In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [3]:
props = pd.read_excel("props.xlsx")

In [30]:
props

,id,barrio,habitaciones,banios,sup_total,sup_cubierta,precio,subte < 1,Asentamiento < 2,Precio M2,...,apto profesional,aire acon,a estrenar,reciclado,a remodelar,dependencia,Escritorio,Apto Credito,En pozo,Luminoso
0,24910,Palermo,4,4,401,400,4000000,1,0,2840,...,0,0,0,0,0,1,0,0,0,0
1,10407,Palermo,5,2,400,200,4000000,0,0,2840,...,0,0,0,0,0,1,1,0,0,0
2,34894,Palermo,4,3,393,378,4000000,1,0,2840,...,0,0,0,0,0,1,1,0,0,0
3,39660,Belgrano,5,4,370,250,4000000,0,0,2700,...,0,1,0,0,0,1,0,0,0,1
4,51305,Palermo,5,3,311,290,4000000,0,0,2840,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36748,17958,Flores,2,1,27,27,30000,1,1,1745,...,0,0,0,0,0,0,1,1,0,0
36749,40235,San Nicolás,1,1,15,15,30000,1,0,1950,...,0,0,0,0,0,0,0,0,0,0
36750,13497,Flores,1,1,38,38,28000,0,0,1745,...,0,0,1,0,0,0,0,1,0,0
36751,41846,Parque Chacabuco,1,1,40,37,70840,1,1,1771,...,0,1,0,0,0,0,1,1,0,0


In [4]:
# Realizo variables dummy para aplicar Xgboost ya que necesita solmanete datos numericos. Tambien se podría haber hecho con OneHotEncoder

dummy = pd.get_dummies(props["barrio"],prefix="barrio")

In [5]:
props = pd.concat([props,dummy],axis=1)

In [6]:
props

,id,barrio,habitaciones,banios,sup_total,sup_cubierta,precio,subte < 1,Asentamiento < 2,Precio M2,...,barrio_Velez Sarsfield,barrio_Versalles,barrio_Villa Crespo,barrio_Villa Devoto,barrio_Villa Ortuzar,barrio_Villa Pueyrredón,barrio_Villa Real,barrio_Villa Santa Rita,barrio_Villa Urquiza,barrio_Villa del Parque
0,24910,Palermo,4,4,401,400,4000000,1,0,2840,...,0,0,0,0,0,0,0,0,0,0
1,10407,Palermo,5,2,400,200,4000000,0,0,2840,...,0,0,0,0,0,0,0,0,0,0
2,34894,Palermo,4,3,393,378,4000000,1,0,2840,...,0,0,0,0,0,0,0,0,0,0
3,39660,Belgrano,5,4,370,250,4000000,0,0,2700,...,0,0,0,0,0,0,0,0,0,0
4,51305,Palermo,5,3,311,290,4000000,0,0,2840,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36748,17958,Flores,2,1,27,27,30000,1,1,1745,...,0,0,0,0,0,0,0,0,0,0
36749,40235,San Nicolás,1,1,15,15,30000,1,0,1950,...,0,0,0,0,0,0,0,0,0,0
36750,13497,Flores,1,1,38,38,28000,0,0,1745,...,0,0,0,0,0,0,0,0,0,0
36751,41846,Parque Chacabuco,1,1,40,37,70840,1,1,1771,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Elimino variable barrio

props.drop(['barrio'],axis=1,inplace=True)

In [18]:
# Renombro 2 variables

props.rename(columns={'Asentamiento < 2':'Asentamiento','subte < 1':'Subte'},inplace=True)

In [42]:
# Realizo el split de los datos. Si quisiera hacer una division lineal del set de datos pondria FALSE en el shuffle.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        props.drop('precio', axis = 'columns'),
                                        props['precio'],
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [44]:
# Parametros y fit del Xgboost 

from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV


xgb1 = XGBRegressor()
parameters = {'nthread':[4], 
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'subsample': [0.5],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 3,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)



Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  18 out of  18 | elapsed:  2.5min finished


[11:26:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[11:26:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=2, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estima...
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None),
             iid='deprecated', n_jobs=3,
             param_grid={'colsample_bytree': [0.5],
                         'learning_rate': [0.03, 0

In [47]:
# Evaluo las predicciones

from sklearn.metrics import mean_squared_error

ypred = xgb_grid.predict(X_test)
prediccion = [round(value) for value in ypred]

meansquarederror =  mean_squared_error(y_test,prediccion)
meansquarederror

8009141221.168549

In [50]:
# Transforomo el MSE en RMSE. EL RMSE esta por debajo de la media que es el baseline en los problemas de regresion, no obstante. La media del precio es 221969

import math

RMSE = math.sqrt(meansquarederror)
RMSE

89493.8054904838

In [55]:
# RMSE del training

from sklearn.metrics import mean_squared_error

ypred = xgb_grid.predict(X_train)
prediccion = [round(value) for value in ypred]

meansquarederror1 =  mean_squared_error(y_train,prediccion)
meansquarederror1

2790303864.5039115

In [72]:
# RMSE del train

import math

RMSE1 = math.sqrt(meansquarederror1)
RMSE1

52823.32689734632

In [63]:
ypred = xgb_grid.predict(X_test)

In [64]:
ypred

array([113116.305, 503751.7  ,  63278.79 , ..., 197231.08 , 187028.17 ,
       355516.38 ], dtype=float32)

In [65]:
ypred = pd.DataFrame(ypred)

In [66]:
ypred

,0
0,113116.304688
1,503751.687500
2,63278.789062
3,156815.921875
4,166650.796875
...,...
7346,158733.578125
7347,174277.953125
7348,197231.078125
7349,187028.171875


In [67]:
# Descargo las predicciones del testing para analizar

ypred.to_excel('pre.xlsx')

In [77]:
X_test

,habitaciones,banios,sup_total,sup_cubierta,Subte,Asentamiento,Precio M2,cochera,pile,baño completo,...,barrio_Velez Sarsfield,barrio_Versalles,barrio_Villa Crespo,barrio_Villa Devoto,barrio_Villa Ortuzar,barrio_Villa Pueyrredón,barrio_Villa Real,barrio_Villa Santa Rita,barrio_Villa Urquiza,barrio_Villa del Parque
25425,2,1,45,42,0,0,1730,1,0,1,...,0,0,0,0,0,0,1,0,0,0
3679,4,2,230,210,1,0,2350,1,0,0,...,0,0,0,0,0,0,0,0,0,0
36510,1,1,25,25,1,0,2002,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16631,2,2,64,52,0,0,1995,0,0,1,...,0,0,0,0,0,0,0,0,0,0
19023,2,1,58,50,1,0,2356,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34351,1,1,36,36,1,0,1950,0,0,1,...,0,0,0,0,0,0,0,0,0,0
32812,1,1,40,40,1,0,2116,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4736,3,2,132,120,1,0,2700,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11654,2,1,51,47,1,0,2840,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# A continuación aplico XGboost con cross validation

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

X_train, X_test, y_train, y_test = train_test_split( props.drop('precio', axis = 'columns'),
                                        props['precio'],
                                        train_size   = 0.2,
                                        random_state = 0,
                                        shuffle      = True)
kf = KFold(n_splits=5)

xgb= XGBRegressor()
xgb.fit(X_train,y_train)

from sklearn.metrics import mean_squared_error

ypred = xgb_grid.predict(X_test)
prediccion2 = [round(value) for value in ypred]

meansquarederror2 =  mean_squared_error(y_test,prediccion2)
meansquarederror2

import math

RMSE2 = math.sqrt(meansquarederror2)
RMSE2

# Se observa que mejora el RMSE realizando validación cruzada en k=5 

62175.327891183544

In [ ]:
# En construccion